# 🔒 FraudGuard Training Notebook

**AD-RL-GNN Fraud Detection** | Full training pipeline with mini-batch processing

This notebook trains the FraudGuard model on the IEEE-CIS fraud detection dataset using:
- **NeighborLoader** for memory-efficient mini-batch training
- **FAISS** for similarity graph construction (GPU if available, CPU fallback)
- **FocalLoss** for class-imbalanced learning

**Target Metrics:**
- Specificity: 98.72%
- G-Means Improvement: 18.11%
- P95 Latency: <100ms

## 1️⃣ Setup Environment

In [ ]:
# Mount Google Drive for data storage
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Clone repository
!git clone https://github.com/govind104/fraudguard.git
%cd fraudguard

Cloning into 'fraudguard'...
remote: Enumerating objects: 154, done.
remote: Counting objects: 100% (154/154), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 154 (delta 60), reused 125 (delta 42), pack-reused 0 (from 0)
Receiving objects: 100% (154/154), 104.58 KiB | 5.23 MiB/s, done.
Resolving deltas: 100% (60/60), done.
/content/fraudguard


In [ ]:
# Install dependencies
# Note: faiss-gpu may not be available on Python 3.12
# The code will fallback to faiss-cpu automatically
# GNN training STILL runs on GPU - only graph building uses CPU FAISS
!pip install -q torch torch-geometric pandas numpy scikit-learn pyyaml structlog

# Try faiss-gpu first, fallback to faiss-cpu
import subprocess
result = subprocess.run(['pip', 'install', '-q', 'faiss-gpu'], capture_output=True)
if result.returncode != 0:
    print('⚠️ faiss-gpu not available, using faiss-cpu')
    print('   (Graph building on CPU, but GNN training still runs on GPU!)')
    !pip install -q faiss-cpu
else:
    print('✓ faiss-gpu installed')

import torch

# 1. Get exact versions
pt_version = torch.__version__.split('+')[0]  # e.g., 2.5.1
cuda_version = "cu" + torch.version.cuda.replace('.', '')  # e.g., cu124
wheel_url = f"https://data.pyg.org/whl/torch-{pt_version}+{cuda_version}.html"

print(f"PyTorch: {pt_version}, CUDA: {cuda_version}")
print(f"Downloading from: {wheel_url}")

# 2. Install with visible output (force reinstall to fix broken partial installs)
!pip install --force-reinstall torch-scatter torch-sparse -f $wheel_url

# Install repo in editable mode
!pip install -e .

print('\n✓ Environment setup complete')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.3 MB/s eta 0:00:00
⚠️ faiss-gpu not available, using faiss-cpu
   (Graph building on CPU, but GNN training still runs on GPU!)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 104.6 MB/s eta 0:00:00
PyTorch: 2.9.0, CUDA: cu126
Looking in links: https://data.pyg.org/whl/torch-2.9.0+cu126.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 16.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.0/35.0 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 133.1 MB/s eta 0:00:00


Obtaining file:///content/fraudguard
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 132.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 6.4 MB/s eta 0:00:00
  Building editable for fraudguard (pyproject.toml) ... done
  Created wheel for fraudguard: filename=fraudguard-0.1.0-py3-none-any.whl size=2801 sha256=5638c0b714acd4b297203f70f102910966a1da4432de3a3f589a22ae37244e19
  Stored in directory: /tmp/pip-ephem-wheel-cache-0ob56ewf/wheels/c6/29/62/fb6d8d095576e7e3efddf4fdcb7dfc799af71ace273f1ee84c
Successfully built fraudguard
  Attempting uninstall: structlog
    Found existing installation: structlog 25.5.0
    Uninstalling structlog-25.5.0:
      Successfully unins


✓ Environment setup complete


In [ ]:
import torch
try:
    import torch_scatter
    import torch_sparse
    import fraudguard
    print("✅ Success! Libraries are installed and loaded.")
except ImportError as e:
    print(f"❌ Still missing libraries: {e}")
    # Only if you see this error should you go back and install again.

✅ Success! Libraries are installed and loaded.


## 2️⃣ Configuration

In [ ]:
import os

# ==============================================
# CONFIGURATION - UPDATE THESE PATHS AS NEEDED
# ==============================================

# Data paths - Point to your Google Drive folders
DATA_DIR = "/content/drive/MyDrive/ieee-fraud-detection"
MODELS_DIR = "/content/drive/MyDrive/fraudguard-models"
LOGS_DIR = "/content/drive/MyDrive/fraudguard-logs"

# Training parameters
SAMPLE_FRAC = 1.0      # Use full dataset (1.0 = 100%)
MAX_EPOCHS = 30
BATCH_SIZE = 4096      # Reduce to 2048 or 1024 if OOM
NUM_NEIGHBORS = [25, 10]  # 2-hop neighborhood sampling

# Create directories
os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(LOGS_DIR, exist_ok=True)

print(f"Data: {DATA_DIR}")
print(f"Models: {MODELS_DIR}")
print(f"Logs: {LOGS_DIR}")
print(f"\nBatch size: {BATCH_SIZE}")
print(f"Sample fraction: {SAMPLE_FRAC*100:.0f}%")

Data: /content/drive/MyDrive/ieee-fraud-detection
Models: /content/drive/MyDrive/fraudguard-models
Logs: /content/drive/MyDrive/fraudguard-logs

Batch size: 4096
Sample fraction: 100%


## 3️⃣ Verify GPU and FAISS

In [ ]:
import torch
import faiss

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print("\n✓ GNN training will run on GPU")
else:
    print("\n⚠️ WARNING: No GPU detected. Go to Runtime > Change runtime type > GPU")

# Check FAISS GPU
faiss_gpus = faiss.get_num_gpus() if hasattr(faiss, 'get_num_gpus') else 0
print(f"\nFAISS GPUs: {faiss_gpus}")
if faiss_gpus == 0:
    print("   (Using CPU FAISS for graph building - this is OK)")

PyTorch version: 2.9.0+cu126
CUDA available: True
GPU: Tesla T4
VRAM: 15.8 GB

✓ GNN training will run on GPU

FAISS GPUs: 0
   (Using CPU FAISS for graph building - this is OK)


## 4️⃣ Load and Preprocess Data

In [ ]:
import sys
sys.path.insert(0, '/content/fraudguard')

from pathlib import Path
from src.data.loader import FraudDataLoader
from src.data.preprocessor import FeaturePreprocessor
from src.data.graph_builder import GraphBuilder
from src.utils.config import load_data_config
from src.utils.device_utils import set_seed, get_device

set_seed(42)
device = get_device()
print(f"Using device: {device}")

# Load config and override path with notebook variable
data_cfg = load_data_config()
data_cfg.paths.raw_data_dir = Path(DATA_DIR)

# Load data with corrected path
loader = FraudDataLoader(config=data_cfg)
df = loader.load_train_data(sample_frac=SAMPLE_FRAC)
train_df, val_df, test_df = loader.create_splits(df)

print(f"\nData loaded:")
print(f"  Train: {len(train_df):,}")
print(f"  Val: {len(val_df):,}")
print(f"  Test: {len(test_df):,}")
print(f"  Fraud rate: {df['isFraud'].mean()*100:.2f}%")

Loading faiss with CPU support (no GPU detected).
Using device: cuda

Data loaded:
  Train: 354,324
  Val: 118,108
  Test: 118,108
  Fraud rate: 3.50%


## 5️⃣ Run Full AD-RL-GNN Pipeline

We use the `FraudTrainer` class to orchestrate the full pipeline, including:
1. **AdaptiveMCD**: Intelligent majority downsampling
2. **RL Agent**: Dynamic subgraph selection (Random Walk, K-Hop, K-Ego)
3. **Graph Enhancement**: Adding semantic edges
4. **GNN Training**: CrossEntropyLoss (15x weight)

In [16]:
import torch
from torch_geometric.data import Data
from torch_geometric.loader import NeighborLoader
from src.training.trainer import FraudTrainer
from src.utils.config import load_model_config, load_data_config
from sklearn.metrics import confusion_matrix
import numpy as np
import time

model_cfg = load_model_config()
data_cfg = load_data_config()
data_cfg.paths.raw_data_dir = Path(DATA_DIR)

# ==============================================================================
# 1. THE MAIN EVENT: 100% DATA
# ==============================================================================
SAMPLE_FRAC = 1.0

# ==============================================================================
# 2. THE GOLD STANDARD CONFIGURATION
# ==============================================================================
model_cfg.training["max_epochs"] = 30
model_cfg.training["learning_rate"] = 0.003
model_cfg.rl_agent["reward_scaling"] = 2.0

# AGGRESSIVE CLEANING (Alpha 0.80)
# Essential for the full dataset to keep the graph manageable
model_cfg.adaptive_mcd["alpha"] = 0.80

# CONNECTIVITY (Threshold 0.75)
# We need this slightly looser threshold so the RL Agent has paths to walk
model_cfg.graph.similarity_threshold = 0.75

# Initialize Trainer
print(f"🚀 Initializing GOLD STANDARD RUN (RL=ON) on {SAMPLE_FRAC*100}% Data...")
trainer = FraudTrainer(
    model_config=model_cfg,
    data_config=data_cfg,
    device=device
)

# ==============================================================================
# 3. FULL SCALE PREP (Expect ~15-20 mins)
# ==============================================================================
print("📉 Loading Full Dataset (590k nodes)...")
loader = FraudDataLoader(config=data_cfg)
df_full = loader.load_train_data(sample_frac=SAMPLE_FRAC)
train_df, val_df, test_df = loader.create_splits(df_full)

print("⚙️ Building Full Graph (Threshold 0.75)...")
trainer._preprocess(train_df, val_df, test_df)
trainer._build_graph()
trainer._prepare_labels(train_df, val_df, test_df)

print("\n🧠 Training AdaptiveMCD (Alpha=0.80)...")
trainer._train_mcd()

# ✅ RL AGENT ENABLED
print("\n🤖 Training RL Agent (Connecting the Dots)...")
trainer._train_rl_and_enhance()

# ==============================================================================
# 4. WEIGHT 12.0
# ==============================================================================
# The "Sweet Spot". We don't need 22.0 because the RL Agent provides the lift.
print("\n⚖️ Applying Sweet Spot Weight 12.0...")
weights = torch.tensor([1.0, 12.0]).to(device)

trainer._init_model()
trainer.criterion = torch.nn.CrossEntropyLoss(weight=weights)
criterion = trainer.criterion
model = trainer.model

# Optimizer
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=0.003,
    weight_decay=1e-4
)

# ==============================================================================
# 5. FULL SCALE TRAINING
# ==============================================================================
print("\n📦 Setting up Loaders (Batch Size 4096)...")
optimized_data = Data(
    x=trainer.X_full,
    edge_index=trainer.edge_index,
    y=trainer.all_labels
)
optimized_data.train_mask = trainer.train_mask
optimized_data.val_mask = trainer.val_mask
optimized_data.test_mask = trainer.test_mask

train_loader = NeighborLoader(
    optimized_data,
    num_neighbors=[25, 10],
    batch_size=4096,
    input_nodes=optimized_data.train_mask,
    shuffle=True
)

val_loader = NeighborLoader(
    optimized_data,
    num_neighbors=[25, 10],
    batch_size=4096,
    input_nodes=optimized_data.val_mask,
    shuffle=False
)

print(f"\n🚀 Starting Full Data Training...")
best_gmeans = 0

for epoch in range(30):
    model.train()
    total_loss = 0
    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index)
        loss = criterion(out[:batch.batch_size], batch.y[:batch.batch_size])

        # Gradient Clipping
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)

    if epoch % 1 == 0:
        model.eval()
        all_preds, all_true = [], []
        with torch.no_grad():
            for batch in val_loader:
                batch = batch.to(device)
                out = model(batch.x, batch.edge_index)
                pred = out[:batch.batch_size].argmax(dim=1)
                all_preds.extend(pred.cpu().numpy())
                all_true.extend(batch.y[:batch.batch_size].cpu().numpy())

        cm = confusion_matrix(all_true, all_preds, labels=[0, 1])
        tn, fp, fn, tp = cm.ravel()
        tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
        tnr = tn / (tn + fp) if (tn + fp) > 0 else 0
        gmeans = np.sqrt(tpr * tnr)

        print(f"Epoch {epoch+1:>3} | Loss: {avg_loss:.4f} | Spec: {tnr*100:.2f}% | Recall: {tpr*100:.2f}% | G-Means: {gmeans*100:.2f}%")

        if gmeans > best_gmeans:
            best_gmeans = gmeans
            # Save as Final Model
            torch.save(model.state_dict(), f"{MODELS_DIR}/fraudguard_gold_standard.pt")

print(f"\nFULL RUN Complete. Best G-Means: {best_gmeans*100:.2f}%")

🚀 Initializing GOLD STANDARD RUN (RL=ON) on 100.0% Data...
📉 Loading Full Dataset (590k nodes)...
⚙️ Building Full Graph (Threshold 0.75)...

🧠 Training AdaptiveMCD (Alpha=0.80)...

🤖 Training RL Agent (Connecting the Dots)...

⚖️ Applying Sweet Spot Weight 12.0...

📦 Setting up Loaders (Batch Size 4096)...


/usr/local/lib/python3.12/dist-packages/torch_geometric/loader/neighbor_loader.py:229: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  neighbor_sampler = NeighborSampler(



🚀 Starting Full Data Training...
Epoch   1 | Loss: 0.7336 | Spec: 88.38% | Recall: 11.58% | G-Means: 31.99%
Epoch   2 | Loss: 0.6669 | Spec: 89.95% | Recall: 10.84% | G-Means: 31.23%
Epoch   3 | Loss: 0.6594 | Spec: 92.62% | Recall: 7.07% | G-Means: 25.59%
Epoch   4 | Loss: 0.6542 | Spec: 94.10% | Recall: 5.83% | G-Means: 23.43%
Epoch   5 | Loss: 0.6477 | Spec: 84.29% | Recall: 17.96% | G-Means: 38.91%
Epoch   6 | Loss: 0.6420 | Spec: 82.55% | Recall: 17.89% | G-Means: 38.43%
Epoch   7 | Loss: 0.6341 | Spec: 77.14% | Recall: 21.19% | G-Means: 40.43%
Epoch   8 | Loss: 0.6254 | Spec: 79.07% | Recall: 21.49% | G-Means: 41.22%
Epoch   9 | Loss: 0.6156 | Spec: 75.57% | Recall: 25.33% | G-Means: 43.75%
Epoch  10 | Loss: 0.6095 | Spec: 71.09% | Recall: 28.67% | G-Means: 45.15%
Epoch  11 | Loss: 0.6051 | Spec: 70.66% | Recall: 29.34% | G-Means: 45.53%
Epoch  12 | Loss: 0.6012 | Spec: 80.43% | Recall: 18.41% | G-Means: 38.48%
Epoch  13 | Loss: 0.5972 | Spec: 79.45% | Recall: 20.58% | G-Means: 

# Vanilla

In [ ]:
import torch
import gc
from torch_geometric.data import Data
from torch_geometric.loader import NeighborLoader
from src.training.trainer import FraudTrainer
from src.utils.config import load_model_config, load_data_config
from sklearn.metrics import confusion_matrix
import numpy as np
import time

# ==============================================================================
# 1. SETUP VANILLA BASELINE (Control Group)
# ==============================================================================
# We disable the "Secret Sauce" to see how a standard model performs.
model_cfg = load_model_config()
data_cfg = load_data_config()
data_cfg.paths.raw_data_dir = Path(DATA_DIR)

# DISABLE INNOVATIONS
model_cfg.adaptive_mcd["alpha"] = 0.0    # Turn off MCD
model_cfg.graph.similarity_threshold = 0.80 # Standard Strict Graph

# Standard Training Params
model_cfg.training["max_epochs"] = 20    # Baseline converges fast
model_cfg.training["learning_rate"] = 0.005

print("🚀 Initializing VANILLA BASELINE (No MCD, No RL)...")
trainer = FraudTrainer(model_config=model_cfg, data_config=data_cfg, device=device)

# Load 100% Data
loader = FraudDataLoader(config=data_cfg)
df_full = loader.load_train_data(sample_frac=1.0)
train_df, val_df, test_df = loader.create_splits(df_full)

# Preprocess
trainer._preprocess(train_df, val_df, test_df)
trainer._build_graph()
trainer._prepare_labels(train_df, val_df, test_df)

# 🚫 SKIP MCD & RL (This is the control group)

# Apply Weight 25.0 (To be fair, we give baseline the same weight help)
weights = torch.tensor([1.0, 25.0]).to(device)
trainer._init_model()
trainer.criterion = torch.nn.CrossEntropyLoss(weight=weights)
model = trainer.model
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=1e-4)

# Loaders
optimized_data = Data(x=trainer.X_full, edge_index=trainer.edge_index, y=trainer.all_labels)
optimized_data.train_mask = trainer.train_mask; optimized_data.val_mask = trainer.val_mask
train_loader = NeighborLoader(optimized_data, num_neighbors=[25, 10], batch_size=4096, input_nodes=optimized_data.train_mask, shuffle=True)
val_loader = NeighborLoader(optimized_data, num_neighbors=[25, 10], batch_size=4096, input_nodes=optimized_data.val_mask, shuffle=False)

print(f"\n📉 Starting Baseline Training...")
best_gmeans_baseline = 0

for epoch in range(20):
    model.train()
    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index)
        loss = trainer.criterion(out[:batch.batch_size], batch.y[:batch.batch_size])
        loss.backward()
        optimizer.step()

    # Eval every 2 epochs
    if epoch % 2 == 0:
        model.eval()
        all_preds, all_true = [], []
        with torch.no_grad():
            for batch in val_loader:
                batch = batch.to(device)
                out = model(batch.x, batch.edge_index)
                pred = out[:batch.batch_size].argmax(dim=1)
                all_preds.extend(pred.cpu().numpy())
                all_true.extend(batch.y[:batch.batch_size].cpu().numpy())

        cm = confusion_matrix(all_true, all_preds, labels=[0, 1])
        tn, fp, fn, tp = cm.ravel()
        tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
        tnr = tn / (tn + fp) if (tn + fp) > 0 else 0
        gmeans = np.sqrt(tpr * tnr)
        print(f"Baseline Epoch {epoch+1:>2} | Spec: {tnr*100:.2f}% | Recall: {tpr*100:.2f}% | G-Means: {gmeans*100:.2f}%")

        if gmeans > best_gmeans_baseline:
            best_gmeans_baseline = gmeans
            torch.save(model.state_dict(), f"{MODELS_DIR}/fraudguard_baseline.pt")

print(f"🏁 Baseline Best G-Means: {best_gmeans_baseline*100:.2f}%")

# Cleanup for Next Run
del model, trainer, optimized_data, train_loader, val_loader
torch.cuda.empty_cache()
gc.collect()

# Improved

In [ ]:
# ==============================================================================
# 2. SETUP AGGRESSIVE GOLD (The Innovation)
# ==============================================================================
model_cfg = load_model_config()

# ✅ MY CHOSEN VALUES (The Logic-Based Selection)
model_cfg.training["max_epochs"] = 30
model_cfg.training["learning_rate"] = 0.003  # Stability for high weights
model_cfg.rl_agent["reward_scaling"] = 2.0
model_cfg.adaptive_mcd["alpha"] = 0.80       # Aggressive cleaning for full data noise
model_cfg.graph.similarity_threshold = 0.75  # Connectivity for RL paths

print(f"\n🚀 Initializing AGGRESSIVE GOLD RUN (RL=ON)...")
trainer = FraudTrainer(model_config=model_cfg, data_config=data_cfg, device=device)

# Re-process (Fast because data is loaded)
trainer._preprocess(train_df, val_df, test_df)
trainer._build_graph()
trainer._prepare_labels(train_df, val_df, test_df)

print("\n🧠 Training AdaptiveMCD (Alpha 0.80)...")
trainer._train_mcd()

print("\n🤖 Training RL Agent (Connecting the dots)...")
trainer._train_rl_and_enhance()

# 🛡️ VRAM FLUSH
print("\n🧹 Flushing VRAM before GNN Training...")
torch.cuda.empty_cache()
gc.collect()

# Weight 25.0 (The Hammer)
weights = torch.tensor([1.0, 25.0]).to(device)
trainer._init_model()
trainer.criterion = torch.nn.CrossEntropyLoss(weight=weights)
model = trainer.model
optimizer = torch.optim.Adam(model.parameters(), lr=0.003, weight_decay=1e-4)

# Loaders (Batch 2048 for safety)
optimized_data = Data(x=trainer.X_full, edge_index=trainer.edge_index, y=trainer.all_labels)
optimized_data.train_mask = trainer.train_mask; optimized_data.val_mask = trainer.val_mask
train_loader = NeighborLoader(optimized_data, num_neighbors=[25, 10], batch_size=2048, input_nodes=optimized_data.train_mask, shuffle=True)
val_loader = NeighborLoader(optimized_data, num_neighbors=[25, 10], batch_size=2048, input_nodes=optimized_data.val_mask, shuffle=False)

print(f"\n🚀 Starting Gold Standard Training...")
best_gmeans_gold = 0

for epoch in range(30):
    model.train()
    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index)
        loss = trainer.criterion(out[:batch.batch_size], batch.y[:batch.batch_size])

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

    if epoch % 1 == 0:
        model.eval()
        all_preds, all_true = [], []
        with torch.no_grad():
            for batch in val_loader:
                batch = batch.to(device)
                out = model(batch.x, batch.edge_index)
                pred = out[:batch.batch_size].argmax(dim=1)
                all_preds.extend(pred.cpu().numpy())
                all_true.extend(batch.y[:batch.batch_size].cpu().numpy())

        cm = confusion_matrix(all_true, all_preds, labels=[0, 1])
        tn, fp, fn, tp = cm.ravel()
        tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
        tnr = tn / (tn + fp) if (tn + fp) > 0 else 0
        gmeans = np.sqrt(tpr * tnr)

        print(f"Epoch {epoch+1:>3} | Spec: {tnr*100:.2f}% | Recall: {tpr*100:.2f}% | G-Means: {gmeans*100:.2f}%")

        if gmeans > best_gmeans_gold:
            best_gmeans_gold = gmeans
            torch.save(model.state_dict(), f"{MODELS_DIR}/fraudguard_final_aggressive.pt")

# Calculate VRAM Savings
peak_mem = torch.cuda.max_memory_allocated() / 1e9
print(f"\n✅ Peak VRAM Usage: {peak_mem:.2f} GB")
print(f"🏁 Final Best G-Means: {best_gmeans_gold*100:.2f}%")

## 6️⃣ Evaluation & Claims Verification

In [ ]:
# Safe Evaluation (Mini-Batch)
print("🔍 Benchmarking & Evaluation...")
model.load_state_dict(torch.load(f"{MODELS_DIR}/fraudguard_best_pro.pt"))
model.eval()

# Use neighbors=[25, 10] for fast inference (Latency < 10ms)
test_loader = NeighborLoader(
    optimized_data,
    num_neighbors=[25, 10],
    batch_size=4096,
    input_nodes=optimized_data.test_mask,
    shuffle=False
)

latencies = []
all_preds, all_true = [], []

with torch.no_grad():
    for batch in test_loader:
        batch = batch.to(device)
        start = time.perf_counter()
        out = model(batch.x, batch.edge_index)
        latencies.append((time.perf_counter() - start) * 1000)
        pred = out[:batch.batch_size].argmax(dim=1)
        all_preds.extend(pred.cpu().numpy())
        all_true.extend(batch.y[:batch.batch_size].cpu().numpy())

# Metrics
cm = confusion_matrix(all_true, all_preds, labels=[0, 1])
tn, fp, fn, tp = cm.ravel()
tnr = tn / (tn + fp) if (tn + fp) > 0 else 0
tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
gmeans = np.sqrt(tpr * tnr)
p95_latency = np.percentile(latencies, 95)

print("="*60)
print("FINAL RESULTS (Pro Framework)")
print("="*60)
print(f"Specificity: {tnr*100:.2f}%")
print(f"Recall:      {tpr*100:.2f}%")
print(f"G-Means:     {gmeans*100:.2f}%")
print(f"P95 Latency: {p95_latency:.2f} ms")

🔍 Benchmarking & Evaluation...


/usr/local/lib/python3.12/dist-packages/torch_geometric/loader/neighbor_loader.py:229: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  neighbor_sampler = NeighborSampler(


FINAL RESULTS (Pro Framework)
Specificity: 63.24%
Recall:      34.47%
G-Means:     46.69%
P95 Latency: 45.51 ms


In [17]:
print("🔍 Benchmarking Gold Standard Model on 100% Test Data...")

# Load Final Model
model.load_state_dict(torch.load(f"{MODELS_DIR}/fraudguard_gold_standard.pt"))
model.eval()

# Full Test Loader
test_loader = NeighborLoader(
    optimized_data,
    num_neighbors=[25, 10],
    batch_size=4096,
    input_nodes=optimized_data.test_mask,
    shuffle=False
)

latencies = []
all_preds, all_true = [], []

with torch.no_grad():
    for batch in test_loader:
        batch = batch.to(device)
        start = time.perf_counter()
        out = model(batch.x, batch.edge_index)
        latencies.append((time.perf_counter() - start) * 1000)

        pred = out[:batch.batch_size].argmax(dim=1)
        all_preds.extend(pred.cpu().numpy())
        all_true.extend(batch.y[:batch.batch_size].cpu().numpy())

cm = confusion_matrix(all_true, all_preds, labels=[0, 1])
tn, fp, fn, tp = cm.ravel()
tnr = tn / (tn + fp) if (tn + fp) > 0 else 0
tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
gmeans = np.sqrt(tpr * tnr)
p95_latency = np.percentile(latencies, 95)

print("="*60)
print("FINAL GOLD STANDARD RESULTS (100% Data, RL=ON)")
print("="*60)
print(f"Confusion Matrix: TP={tp}, TN={tn}, FP={fp}, FN={fn}")
print(f"Specificity: {tnr*100:.2f}%")
print(f"Recall:      {tpr*100:.2f}%")
print(f"G-Means:     {gmeans*100:.2f}%")
print(f"P95 Latency: {p95_latency:.2f} ms")

🔍 Benchmarking Gold Standard Model on 100% Test Data...


/usr/local/lib/python3.12/dist-packages/torch_geometric/loader/neighbor_loader.py:229: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  neighbor_sampler = NeighborSampler(


FINAL GOLD STANDARD RESULTS (100% Data, RL=ON)
Confusion Matrix: TP=1279, TN=80397, FP=33647, FN=2785
Specificity: 70.50%
Recall:      31.47%
G-Means:     47.10%
P95 Latency: 45.49 ms


In [ ]:
# ==============================================================================
# 3. FINAL A/B TEST REPORT
# ==============================================================================
print("\n" + "="*60)
print("🎯 FINAL ARCHITECTURAL COMPARISON")
print("="*60)
print(f"Baseline G-Means (No RL/MCD): {best_gmeans_baseline*100:.2f}%")
print(f"System   G-Means (AD-RL-GNN): {best_gmeans_gold*100:.2f}%")

improvement = best_gmeans_gold - best_gmeans_baseline
pct_improvement = (improvement / best_gmeans_baseline) * 100

print(f"------------------------------------------------------------")
print(f"🚀 ABSOLUTE IMPROVEMENT: +{improvement*100:.2f}% points")
print(f"🚀 RELATIVE IMPROVEMENT: +{pct_improvement:.2f}%")
print("="*60)

## 7️⃣ Save Model

In [ ]:
trainer.save(f"{MODELS_DIR}/fraudguard_full_pipeline.pt")
print(f"Model saved to {MODELS_DIR}/fraudguard_full_pipeline.pt")

Model saved to /content/drive/MyDrive/fraudguard-models/fraudguard_full_pipeline.pt


In [ ]:
from google.colab import runtime
runtime.unassign()